In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

## LOAD - Operations

In [3]:
## Load Operations 

operations_df = pd.read_csv('../_data/operation_pcd.csv')
operations_df.columns, operations_df.shape, operations_df.info()

test = operations_df[operations_df['subject_id']==121939743]
test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128031 entries, 0 to 128030
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   op_id              128031 non-null  int64  
 1   subject_id         128031 non-null  int64  
 2   hadm_id            128031 non-null  int64  
 3   opdate             128031 non-null  int64  
 4   age                128031 non-null  int64  
 5   sex                128031 non-null  object 
 6   weight             126611 non-null  float64
 7   height             127269 non-null  float64
 8   race               128031 non-null  object 
 9   asa                124636 non-null  float64
 10  emop               128031 non-null  int64  
 11  department         128031 non-null  object 
 12  antype             128031 non-null  object 
 13  icd10_pcs          128031 non-null  object 
 14  category_desc      128031 non-null  object 
 15  desc_short         128031 non-null  object 
 16  ca

,op_id,subject_id,hadm_id,opdate,age,sex,weight,height,race,asa,...,opend_time,admission_time,discharge_time,anstart_time,anend_time,cpbon_time,cpboff_time,icuin_time,icuout_time,inhosp_death_time
4911,484032217,121939743,273337113,14400,60,M,70.0,179.0,Asian,2.0,...,15640.0,0,136795,14880.0,15630.0,NaN,NaN,15645.0,16895.0,NaN
5213,494869962,121939743,273337113,27360,60,M,70.0,179.0,Asian,2.0,...,28660.0,0,136795,28540.0,28660.0,NaN,NaN,NaN,NaN,NaN
5968,437375846,121939743,273337113,59040,60,M,70.0,179.0,Asian,2.0,...,59700.0,0,136795,59530.0,59715.0,NaN,NaN,NaN,NaN,NaN


## Vitals
### LOAD - Vitals

In [4]:
## Load VITALS
vitals_df = pd.read_csv('../_data/vitals_in_hospital_filter.csv')
vitals_df.head()

test_vital = vitals_df[vitals_df['op_id']==494869962]
test_vital

,#,op_id,subject_id,chart_time,item_name,value,nearest_orout
160453,160454,494869962,121939743,28655,hr,100.0,28670
160454,160455,494869962,121939743,28655,pip,26.0,28670
160455,160456,494869962,121939743,28655,rr,22.5,28670
160456,160457,494869962,121939743,28655,spo2,100.0,28670
160457,160458,494869962,121939743,28655,vt,296.0,28670


### Pivot - Vitals

In [5]:
pivoted_vitals = pd.pivot_table(vitals_df, index=['op_id', 'subject_id', 'chart_time'], columns='item_name', values='value')
pivoted_vitals.head()
pivoted_vitals.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 128491 entries, (400000455, 179458020, 2005) to (499999032, 136003154, 2175)
Data columns (total 20 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   art_dbp  36721 non-null   float64
 1   art_mbp  37555 non-null   float64
 2   art_sbp  36808 non-null   float64
 3   bis      1963 non-null    float64
 4   bt       27719 non-null   float64
 5   ci       53 non-null      float64
 6   cvp      2691 non-null    float64
 7   ffp      230 non-null     float64
 8   ftn      94 non-null      float64
 9   hr       112146 non-null  float64
 10  pap_dbp  322 non-null     float64
 11  pap_mbp  366 non-null     float64
 12  pap_sbp  325 non-null     float64
 13  pip      83815 non-null   float64
 14  pmean    79294 non-null   float64
 15  rbc      1011 non-null    float64
 16  rr       98032 non-null   float64
 17  spo2     116990 non-null  float64
 18  uo       4155 non-null    float64
 19  vt       85338 n

As a result of the pivot, these columns were converted into the index:  
- 0   #              5 non-null      int64  
- 1   **op_id**          5 non-null      int64  
- 2   **subject_id**     5 non-null      int64  
- 3   **chart_time**     5 non-null      int64  

Need to reset the index to the table can be used normallly again. 

In [6]:
# confirm that the index changed to the above: 
pivoted_vitals.index

#Reset the index! 
pivoted_vitals.reset_index(inplace=True)

# confim again. 
pivoted_vitals.index

RangeIndex(start=0, stop=128491, step=1)

In [7]:

## Test pivoted_vitals = ensure there remains only 1 record of an op_id
test_ppivot = pivoted_vitals[pivoted_vitals['op_id']==494869962]
test_ppivot


item_name,op_id,subject_id,chart_time,art_dbp,art_mbp,art_sbp,bis,bt,ci,cvp,...,pap_dbp,pap_mbp,pap_sbp,pip,pmean,rbc,rr,spo2,uo,vt
121792,494869962,121939743,28655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26.0,NaN,NaN,22.5,100.0,NaN,296.0


### Preliminary EDA - VITALS
1. Drop fields with high null count. 

In [8]:
vitals_todrop = ['bis','ci', 'ffp','ftn','pap_dbp','pap_mbp','pap_sbp','rbc','uo']
pivoted_vitals.drop(columns=vitals_todrop, inplace=True)
pivoted_vitals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128491 entries, 0 to 128490
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   op_id       128491 non-null  int64  
 1   subject_id  128491 non-null  int64  
 2   chart_time  128491 non-null  int64  
 3   art_dbp     36721 non-null   float64
 4   art_mbp     37555 non-null   float64
 5   art_sbp     36808 non-null   float64
 6   bt          27719 non-null   float64
 7   cvp         2691 non-null    float64
 8   hr          112146 non-null  float64
 9   pip         83815 non-null   float64
 10  pmean       79294 non-null   float64
 11  rr          98032 non-null   float64
 12  spo2        116990 non-null  float64
 13  vt          85338 non-null   float64
dtypes: float64(11), int64(3)
memory usage: 13.7 MB


## LABS
### Load - Labs

In [9]:
## Load LABS
labs_df = pd.read_csv('../_data/labs_in_hospital_filter.csv')
labs_df.columns, labs_df.shape

(Index(['#', 'subject_id', 'chart_time', 'item_name', 'value', 'nearest_orout'], dtype='object'),
 (629055, 6))

### Pivot - Labs

In [10]:
pivoted_labs = pd.pivot_table(labs_df, index=['subject_id', 'chart_time'], columns='item_name', values='value')
pivoted_labs.head()

item_name               alp   alt   ast  chloride  creatinine  crp  glucose  \
subject_id chart_time                                                         
100001820  20595        NaN   NaN   NaN       NaN         NaN  NaN    167.0   
100002094  3615        45.0  16.0  21.0     113.0        0.50  NaN      NaN   
           3528960      NaN   NaN   NaN       NaN         NaN  NaN    100.0   
100002234  2425         NaN   NaN   NaN       NaN        0.95  NaN      NaN   
100002413  1165         NaN   NaN   NaN       NaN        0.85  NaN      NaN   

item_name                hb  hba1c  hco3  lymphocyte  platelet  potassium  \
subject_id chart_time                                                       
100001820  20595        NaN    NaN   NaN         NaN       NaN        NaN   
100002094  3615        13.6    NaN  24.4         5.1     184.0        3.3   
           3528960      NaN    NaN   NaN         NaN       NaN        NaN   
100002234  2425         NaN    NaN   NaN         NaN       NaN        NaN   
100002413  1165         NaN    NaN   NaN         NaN       NaN        NaN   

item_name              sodium  total_bilirubin  wbc  
subject_id chart_time                                
100001820  20595          NaN              NaN  NaN  
100002094  3615         145.0              1.0  9.1  
           3528960        NaN              NaN  NaN  
100002234  2425           NaN              NaN  NaN  
100002413  1165           NaN              NaN  NaN

In [11]:
# confirm that the index changed to the above: 
pivoted_labs.index


MultiIndex([(100001820,   20595),
            (100002094,    3615),
            (100002094, 3528960),
            (100002234,    2425),
            (100002413,    1165),
            (100004062,  156050),
            (100004062,  166150),
            (100004062,  195625),
            (100010304,     520),
            (100010304,    1125),
            ...
            (199994133,    2235),
            (199994133,  919860),
            (199994802,    1935),
            (199994802,  488990),
            (199995430,    6905),
            (199997812,    1245),
            (199998113,    2035),
            (199998113,  170795),
            (199999332,    2780),
            (199999413,    3855)],
           names=['subject_id', 'chart_time'], length=95322)

In [12]:
#Reset the index! 
pivoted_labs.reset_index(inplace=True)

# confim again. 
pivoted_labs.index



RangeIndex(start=0, stop=95322, step=1)

In [13]:
# Testing: Because Labs does not have an OP_ID, we use Subject id instead. subject == 121939743

## Test pivoted_vitals = ensure there remains only 1 record of an op_id
testv_ppivot = pivoted_labs[pivoted_labs['subject_id']==121939743]
testv_ppivot

item_name,subject_id,chart_time,alp,alt,ast,chloride,creatinine,crp,glucose,hb,hba1c,hco3,lymphocyte,platelet,potassium,sodium,total_bilirubin,wbc
20725,121939743,15665,38.0,46.0,89.0,111.0,1.41,NaN,NaN,12.8,NaN,18.1,7.2,182.0,5.0,140.0,3.7,7.8
20726,121939743,28675,133.0,33.0,50.0,103.0,0.97,8.65,NaN,9.1,NaN,24.3,NaN,337.0,4.0,136.0,0.8,9.1
20727,121939743,59730,85.0,74.0,58.0,100.0,1.00,NaN,NaN,9.7,NaN,25.8,31.5,182.0,4.3,133.0,0.9,8.3


### Preliminary EDA - Labs
1. Drop fields with high NA

In [14]:
pivoted_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95322 entries, 0 to 95321
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   subject_id       95322 non-null  int64  
 1   chart_time       95322 non-null  int64  
 2   alp              34769 non-null  float64
 3   alt              36429 non-null  float64
 4   ast              36448 non-null  float64
 5   chloride         44877 non-null  float64
 6   creatinine       40262 non-null  float64
 7   crp              9170 non-null   float64
 8   glucose          46707 non-null  float64
 9   hb               46898 non-null  float64
 10  hba1c            507 non-null    float64
 11  hco3             32364 non-null  float64
 12  lymphocyte       36306 non-null  float64
 13  platelet         44989 non-null  float64
 14  potassium        63067 non-null  float64
 15  sodium           62797 non-null  float64
 16  total_bilirubin  33308 non-null  float64
 17  wbc         

In [15]:
labs_todrop = ['crp','hba1c']
pivoted_labs.drop(columns=labs_todrop, inplace=True)
pivoted_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95322 entries, 0 to 95321
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   subject_id       95322 non-null  int64  
 1   chart_time       95322 non-null  int64  
 2   alp              34769 non-null  float64
 3   alt              36429 non-null  float64
 4   ast              36448 non-null  float64
 5   chloride         44877 non-null  float64
 6   creatinine       40262 non-null  float64
 7   glucose          46707 non-null  float64
 8   hb               46898 non-null  float64
 9   hco3             32364 non-null  float64
 10  lymphocyte       36306 non-null  float64
 11  platelet         44989 non-null  float64
 12  potassium        63067 non-null  float64
 13  sodium           62797 non-null  float64
 14  total_bilirubin  33308 non-null  float64
 15  wbc              44500 non-null  float64
dtypes: float64(14), int64(2)
memory usage: 11.6 MB


## MERGE 
### Operations_pcd with Vitals, then Labs.

In [16]:
operations_w_vitals_df = pd.merge(operations_df,pivoted_vitals, on='op_id', how='left')
operations_w_vitals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128494 entries, 0 to 128493
Data columns (total 43 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   op_id              128494 non-null  int64  
 1   subject_id_x       128494 non-null  int64  
 2   hadm_id            128494 non-null  int64  
 3   opdate             128494 non-null  int64  
 4   age                128494 non-null  int64  
 5   sex                128494 non-null  object 
 6   weight             127068 non-null  float64
 7   height             127726 non-null  float64
 8   race               128494 non-null  object 
 9   asa                124952 non-null  float64
 10  emop               128494 non-null  int64  
 11  department         128494 non-null  object 
 12  antype             128494 non-null  object 
 13  icd10_pcs          128494 non-null  object 
 14  category_desc      128494 non-null  object 
 15  desc_short         128494 non-null  object 
 16  ca

In [17]:
operations_w_vitals_df.rename(columns={'subject_id_x': 'subject_id'}, inplace=True)
operations_w_vitals_df[operations_w_vitals_df['op_id']==494869962]

,op_id,subject_id,hadm_id,opdate,age,sex,weight,height,race,asa,...,art_mbp,art_sbp,bt,cvp,hr,pip,pmean,rr,spo2,vt
5230,494869962,121939743,273337113,27360,60,M,70.0,179.0,Asian,2.0,...,NaN,NaN,NaN,NaN,100.0,26.0,NaN,22.5,100.0,296.0


In [18]:
operations_vitals_labs_df = pd.merge(operations_w_vitals_df,pivoted_labs, on='subject_id', how='left')

operations_vitals_labs_df.shape, operations_vitals_labs_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200593 entries, 0 to 200592
Data columns (total 58 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   op_id              200593 non-null  int64  
 1   subject_id         200593 non-null  int64  
 2   hadm_id            200593 non-null  int64  
 3   opdate             200593 non-null  int64  
 4   age                200593 non-null  int64  
 5   sex                200593 non-null  object 
 6   weight             198158 non-null  float64
 7   height             199124 non-null  float64
 8   race               200593 non-null  object 
 9   asa                194133 non-null  float64
 10  emop               200593 non-null  int64  
 11  department         200593 non-null  object 
 12  antype             200593 non-null  object 
 13  icd10_pcs          200593 non-null  object 
 14  category_desc      200593 non-null  object 
 15  desc_short         200593 non-null  object 
 16  ca

((200593, 58), None)

## Export to CSV
### NOTES
* `operations_vitals_labs_df`contains duplicates due to joining LABS data. (no unique primary key between operations and labs)
* `operations_full_nodup` has had all duplicates (defined by: op_id / subject_id / hadm_id)

In [19]:
# EXPORT and Specify the file path 
output_csv_file = '../_data/operations_fulldata.csv'

# Save the DataFrame to a CSV file
operations_vitals_labs_df.to_csv(output_csv_file, index=False)

In [20]:
operations_full_nodup = operations_vitals_labs_df.drop_duplicates(subset=['op_id', 'subject_id', 'hadm_id'], keep='first')
operations_full_nodup

# EXPORT and Specify the file path 
output_csv_file = '../_data/operations_fulldata_nodup.csv'

# Save the DataFrame to a CSV file
operations_full_nodup.to_csv(output_csv_file, index=False)